In [6]:
import numpy as np
import cv2
import glob

gridDim = 7
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((gridDim*gridDim,3), np.float32)

objp[:,:2] = np.mgrid[0:gridDim,0:gridDim].T.reshape(-1,2)


'''
cell size is 22 millimeters
so scaling a matrix in order
to have a millimeter values 
of intrinsic and extrinsic
 parameters
'''
#objp *= 22

print(objp)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('/home/wil/calibrationPhotos/*.jpg')
counter = 0
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (gridDim,gridDim),None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        counter += 1
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (gridDim,gridDim), corners2,ret)
        img = cv2.resize(img, (540, 960))  
        cv2.imshow(fname,img)
        cv2.waitKey(500)

cv2.destroyAllWindows()
print(counter, " of correct images from ", len(images))

import os
duration = 10  # seconds
freq = 440  # Hz
os.system('play -nq -t alsa synth {} sine {}'.format(duration, freq))

[[0. 0. 0.]
 [1. 0. 0.]
 [2. 0. 0.]
 [3. 0. 0.]
 [4. 0. 0.]
 [5. 0. 0.]
 [6. 0. 0.]
 [0. 1. 0.]
 [1. 1. 0.]
 [2. 1. 0.]
 [3. 1. 0.]
 [4. 1. 0.]
 [5. 1. 0.]
 [6. 1. 0.]
 [0. 2. 0.]
 [1. 2. 0.]
 [2. 2. 0.]
 [3. 2. 0.]
 [4. 2. 0.]
 [5. 2. 0.]
 [6. 2. 0.]
 [0. 3. 0.]
 [1. 3. 0.]
 [2. 3. 0.]
 [3. 3. 0.]
 [4. 3. 0.]
 [5. 3. 0.]
 [6. 3. 0.]
 [0. 4. 0.]
 [1. 4. 0.]
 [2. 4. 0.]
 [3. 4. 0.]
 [4. 4. 0.]
 [5. 4. 0.]
 [6. 4. 0.]
 [0. 5. 0.]
 [1. 5. 0.]
 [2. 5. 0.]
 [3. 5. 0.]
 [4. 5. 0.]
 [5. 5. 0.]
 [6. 5. 0.]
 [0. 6. 0.]
 [1. 6. 0.]
 [2. 6. 0.]
 [3. 6. 0.]
 [4. 6. 0.]
 [5. 6. 0.]
 [6. 6. 0.]]
16  of correct images from  47


512

In [2]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)
print(ret)
print(mtx)
print(dist)
print(rvecs)
print(tvecs)


2.191732377536117
[[3.04172433e+03 0.00000000e+00 1.17949297e+03]
 [0.00000000e+00 3.09282061e+03 2.28600971e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
[[ 0.02941472  0.23838415  0.0433189   0.00735723 -0.74572005]]
[array([[ 0.0503803 ],
       [-0.15939099],
       [ 0.57936423]]), array([[-0.56273432],
       [-0.36177075],
       [-0.17166943]]), array([[-0.51996806],
       [-0.25164854],
       [ 0.25905397]]), array([[ 0.35783542],
       [-0.10949941],
       [ 0.79123153]]), array([[-0.6709151 ],
       [-0.47706486],
       [ 0.26414602]]), array([[-0.78465809],
       [ 0.57404156],
       [ 2.00689749]]), array([[-0.57877225],
       [ 0.55730629],
       [ 1.45452093]]), array([[ 0.2190016 ],
       [-0.31219439],
       [ 1.15249299]]), array([[-0.85244373],
       [-0.47267052],
       [-0.15199807]]), array([[-0.22702911],
       [-0.04092067],
       [ 0.69451615]]), array([[-0.3399079 ],
       [ 0.02570317],
       [ 0.59511278]]), array([[-0.31885523],
  

In [5]:
print(len(rvecs[0]))

3


In [7]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)
print(ret)
print(mtx)
print(dist)
print(rvecs)
print(tvecs)

2.1917323775361135
[[3.04172433e+03 0.00000000e+00 1.17949297e+03]
 [0.00000000e+00 3.09282061e+03 2.28600970e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
[[ 0.02941471  0.23838426  0.0433189   0.00735723 -0.74572031]]
[array([[ 0.0503803 ],
       [-0.15939099],
       [ 0.57936423]]), array([[-0.56273432],
       [-0.36177075],
       [-0.17166943]]), array([[-0.51996806],
       [-0.25164854],
       [ 0.25905397]]), array([[ 0.35783542],
       [-0.10949941],
       [ 0.79123153]]), array([[-0.6709151 ],
       [-0.47706486],
       [ 0.26414602]]), array([[-0.78465809],
       [ 0.57404156],
       [ 2.00689749]]), array([[-0.57877225],
       [ 0.5573063 ],
       [ 1.45452093]]), array([[ 0.2190016 ],
       [-0.31219439],
       [ 1.15249299]]), array([[-0.85244373],
       [-0.47267052],
       [-0.15199807]]), array([[-0.22702911],
       [-0.04092067],
       [ 0.69451615]]), array([[-0.3399079 ],
       [ 0.02570317],
       [ 0.59511278]]), array([[-0.31885523],
 

In [ ]:
img = cv2.imread('left12.jpg')
h,  w = img.shape[:2]
newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))


In [ ]:
# undistort
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

# crop the image
x,y,w,h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('calibresult.png',dst)